# Quality Assessment
Now it's your turn. Follow [the steps on the platform](https://learn.wbscodingschool.com/courses/data-science/lessons/%f0%9f%93%9d-5-data-quality/) and use what you've learnt to see how reliable the data is.

In [2]:
import pandas as pd

Load our cleaned DataFrames

In [3]:
# orders_cl.csv
url = "https://drive.google.com/file/d/1Tla62vfu__kCqvgypZyVt2S9VuC016yH/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_csv(path)

# orderlines_cl.csv
url = "https://drive.google.com/file/d/1OhtkQS2fwOYdzfd-qPh7im35iLc-L9TA/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_csv(path)

# products_cl.csv
url = "https://drive.google.com/file/d/1s7Lai4NSlsYjGEPg1QSOUJobNYVsZBOJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)

In [4]:
orders_df = orders_cl.copy()
orderlines_df = orderlines_cl.copy()
products_df = products_cl.copy()

## 1.&nbsp; Define Pandas display format

In [5]:
# your code here
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)

## 2.&nbsp; Exclude unwanted orders

In [6]:
# your code here
order_state_mask = (orders_df['state'] == 'Cancelled') | (orders_df['state'] == 'Shopping Basket')
orders_df = orders_df.loc[~order_state_mask].copy()

orderlines_df = orderlines_df.loc[orderlines_df['id_order'].isin(orders_df['order_id'])]

orders_df = orders_df.loc[orders_df['order_id'].isin(orderlines_df['id_order'])]

## 3.&nbsp; Exclude orders with unknown products


In [7]:
# your code here
orderlines_df = orderlines_df.loc[orderlines_df['sku'].isin(products_df['sku'])]
orders_df = orders_df.loc[orders_df['order_id'].isin(orderlines_df['id_order'])]

## 4.&nbsp; Explore the revenue from different tables

#### Step 1:
Create the `unit_price_total` as `orderlines.unit_price` * `orderlines.product_quantity`

In [8]:
# your code here
orderlines_df['unit_price_total'] = orderlines_df['unit_price'].copy() * orderlines_df['product_quantity'].copy()

#### Step 2:
Group by `id_order`, summarising by the sum of `unit_price_total`

In [9]:
# your code here
orderlines_grouped = orderlines_df.groupby('id_order')['unit_price_total'].sum()
orders_df = orders_df.merge(orderlines_grouped, left_on='order_id', right_on='id_order', how='inner')
orders_df

,order_id,created_date,total_paid,state,unit_price_total
0,241423,2017-11-06 13:10:02,136.15,Completed,129.16
1,242832,2017-12-31 17:40:03,15.76,Completed,10.77
2,243330,2017-02-16 10:59:38,84.98,Completed,77.99
3,245275,2017-06-28 11:35:37,149.00,Completed,149.00
4,245595,2017-01-21 12:52:47,112.97,Completed,105.98
...,...,...,...,...,...
78185,527392,2018-03-14 13:55:11,34.99,Pending,34.99
78186,527394,2018-03-14 13:55:15,161.00,Place Order,161.00
78187,527396,2018-03-14 13:55:44,141.58,Place Order,141.58
78188,527397,2018-03-14 13:56:38,42.99,Place Order,42.99


### What is the average difference between `total_paid` and `unit_price_total`?

In [10]:
# your code here
average_difference_total_unit = (orders_df['total_paid'] - orders_df['unit_price_total']).mean()
average_difference_total_unit

4.898604936692671

In [11]:
orders_df

,order_id,created_date,total_paid,state,unit_price_total
0,241423,2017-11-06 13:10:02,136.15,Completed,129.16
1,242832,2017-12-31 17:40:03,15.76,Completed,10.77
2,243330,2017-02-16 10:59:38,84.98,Completed,77.99
3,245275,2017-06-28 11:35:37,149.00,Completed,149.00
4,245595,2017-01-21 12:52:47,112.97,Completed,105.98
...,...,...,...,...,...
78185,527392,2018-03-14 13:55:11,34.99,Pending,34.99
78186,527394,2018-03-14 13:55:15,161.00,Place Order,161.00
78187,527396,2018-03-14 13:55:44,141.58,Place Order,141.58
78188,527397,2018-03-14 13:56:38,42.99,Place Order,42.99


### What is the distribution of these differences?

In [12]:
# your code here
orders_df['created_date'] = pd.to_datetime(orders_df['created_date'])
orders_df['difference_total_unit'] = orders_df['total_paid'] - orders_df['unit_price_total']
(
orders_df['difference_total_unit']
    .resample("M", on="created_date")
    .mean()
    .plot()
 );

KeyError: 'created_date'

### Can all the differences be explained by shipping costs? If not, what are other plausible explanations?

In [ ]:
# your explanation here
len(orders_df['difference_total_unit'].unique())

997

### If there are differences that you can’t explain: what should you do with these orders?

In [ ]:
# your code here
difference_mask = orders_df['difference_total_unit'] < 0 
difference_mask.sum()

1433

## 5.&nbsp; Become confident about your dataset

Let's hear your thoughts about what you discovered.

In [ ]:
# your thoughts & code here

Do not forget to download your quality controlled DataFrames

In [ ]:
#from google.colab import files

orders_df.to_csv("./clean_data/orders_qu.csv", index=False)
#files.download("orders_qu.csv")

orderlines_df.to_csv("./clean_data/orderlines_qu.csv", index=False)
#files.download("orderlines_qu.csv")

products_df.to_csv("./clean_data/products_qu.csv", index=False)